Note to user:
 - The following is a summary of findings using a small subset of faculty/researchers from the University of Cincinnati

First we discovered that Dimensions can store individuals under multiple Researcher IDs, making a little legwork necessary to verify they are all the same person

Additionally, the researchers in question can be stored in the database with different first names. By this I mean some may have a preferred name, a middle initial with their first name, or possibly even only their first initial

Another finding is that Dimensions is not immediate in logging publications to a given researcher ID, they have to verify it is the same person, and failing to do so leads to the existance of multiple IDs. This also means that some publications will not be found because we are searching for only UC affiliations and some of these temporary identities is missing the affiliation to UC. We can be confident that anything from before a year ago is logged with the appropriate affiliations. Following that, the next 6 months are less certain and anything recently published (around a month or two) will not be in the system at all.

## Step 1: Set up the dimensions API and required imports and log in

Method 1:
- use input() to ask for the key for you to paste in each time

In [ ]:
!pip install dimcli -U --quiet

import dimcli
from dimcli.utils import *
import json
import sys
import pandas as pd
import re
import numpy as np

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
  import getpass
  KEY = input("Input your API key for this session: ")
  dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
  KEY = ""
  dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()

Method 2:
- save the key as a Secret directly to google Colab.

The Notebook will ask for permission to access it and once granted will use the saved value

To create a Secret, click the sideways key on the left side of the screen (its below {x} and above the folder icon)

Once there give your secret a name such as Dimension_API_Key or one of your choosing

Assign the value as your API key (dont add quotation marks)

With that, your key is now saved to your google account to be used whenever.

In [ ]:
!pip install dimcli -U --quiet

import dimcli
from dimcli.utils import *
import json
import sys
import pandas as pd
import re
import numpy as np

print("==\nLogging in..")
from google.colab import userdata
dimcli.login(key=userdata.get('DimensionKey'), endpoint="https://app.dimensions.ai")
dsl = dimcli.Dsl()

First, install the dimcli package to interact with the Dimensions API.
- This command installs the `dimcli` package, which provides the necessary tools to interact with the Dimensions database. The `-U` flag ensures that you install or update to the latest version, and the `--quiet` flag suppresses unnecessary output during installation.

Second, we will import the libraries and modules necessary for using the Dimensions API.
- `dimcli`: The library used to interact with the Dimensions API.
- `json`: A library for parsing JSON data returned by the API.
- `sys`: Used for handling system-specific parameters, like checking if the notebook is running in Google Colab.
- `pandas`: A powerful data manipulation library, useful for handling and analyzing datasets.
- `numpy`: A library for numerical operations, which can be helpful for efficient data manipulation, calculations, or creating arrays.

Third, to use the Dimensions API, you need to authenticate with your personal API key. In this step, you'll input your API key to log in.
- The script checks if it’s running in Google Colab using `sys.modules`.
- If it is in Google Colab, it will prompt you to input your API key using `input()`. You can securely enter the key in Colab.
- The `dimcli.login()` function authenticates with the API using the provided API key.
- The `Dsl()` function creates an instance of the Dimensions API's Domain-Specific Language (DSL), which will be used to query the database.

## Step 2: Load Faculty Data and Prepare Lists of Faculty Members
In this step, we will read an Excel file containing the faculty roster and organize the data into dictionaries and lists that will be used for further analysis.

We start by reading the Excel file that contains the faculty data. This file includes data for various years across different sheets.
- The `pd.read_excel()` function loads the Excel file `'Faculty Roster_Pillay Request_11.06.2024.xlsx'`.
- The `['11.01.2018', '11.01.2019', ...]` argument specifies the list of sheet names to read from the Excel file, which correspond to different years.
- The result is stored in `faculty_data`, a dictionary where each key is a year (e.g., `'11.01.2018'`), and the corresponding value is the data for that year in the form of a DataFrame.


In [ ]:
!wget 'https://raw.githubusercontent.com/The-CEAS-Library/Dimensions-API-Querying/master/Faculty%20Roster_Pillay%20Request_11.06.2024.xlsx' -O Faculty_Roster.xlsx
faculty_data = pd.read_excel('Faculty_Roster.xlsx',['11.01.2018', '11.01.2019', '11.01.2020','11.01.2021','11.01.2022','11.01.2023','11.01.2024'])



# Next, we initialize empty lists to store the faculty members' names in different formats.
fullName, fName, lName, empNameAgg = [], [], [], []
# fullName: A list for storing the full name of each faculty member (first and last).
# fName: A list for storing the first names of faculty members.
# lName: A list for storing the last names of faculty members.
# empNameAgg: An aggregated list of all unique names across years.



# We now extract and clean the employee names for each year and store them in a dictionary.
# The names are cleaned by removing any periods, and the resulting list is sorted.

# List of years for each date
years = ['2018', '2019', '2020', '2021', '2022', '2023', '2024']

# Gather and clean employee names by year
#
empNames = {f'{year}': sorted(name.replace('.', '') for name in faculty_data[f'11.01.{year}']['Employee']) for year in years }

# empNames: A dictionary where each key is a year (e.g., `'2018'`), and the value is a sorted list of employee names after cleaning.
# The names are cleaned by removing periods, (`name.replace('.', '')`), because the database does not contain them in the names.



#Next, we aggregate all unique faculty names from each year into one sorted list.
empNameAgg = sorted(set(name for names in empNames.values() for name in names))

# This step flattens all the name lists across years and removes duplicates using set().
# The result is a sorted list of unique faculty member names stored in empNameAgg.

# We will now extract and organize the first names, last names, and full names from all the sheets in faculty_data.

# Extract and organize name components across all sheets in faculty_data
for sheet_data in faculty_data.values():
    fName.extend(sheet_data['Preferred First Name'])
    lName.extend(sheet_data['Preferred Last Name'])
    fullName.extend(f"{last} {first}" for first, last in zip(sheet_data['Preferred First Name'], sheet_data['Preferred Last Name']))

# if your excel sheet uses different column names, you will need to change the values inside the [] to match
# For each sheet in faculty_data, we extract the 'Preferred First Name' and 'Preferred Last Name' columns.
# We then create the fullName list by combining first and last names into one string.
# The fName, lName, and fullName lists are extended with the respective data from each sheet.


# Put all the full names together into the single list and remove duplicates as well as missing info (in our case the # symbol)
empNameAgg.extend(fullName)
empNameAgg = sorted(set(empNameAgg))

for name in empNameAgg:
  if '#' in name:
    empNameAgg.remove(name)

# list of unique last names
uniqueLN = list(set(lName))

In [ ]:
#optional route for missing data (this grabs the people who are missing affiliations)
#doing it this way will take a long time (for our small set of data it was ~45mins)

emplast_names = [name.split(" ", 1)[0] for name in empNameAgg]
empfirst_names = [name.split(" ", 1)[1] for name in empNameAgg]


## Step 3: Retrieve Researcher IDs from the Dimensions Database
In this step, we will query the Dimensions database to retrieve information about researchers based on their last names and the research organization they belong to (in this case, the University of Cincinnati).

### Step 3.1: Define the Grid ID for the University of Cincinnati
To query researchers at the University of Cincinnati, we first define the Grid ID for the university.
- The `GRIDID` is the unique identifier for the University of Cincinnati in the Dimensions database. You will use this ID in the query to filter results by the university.

In [ ]:
# method 1

# this method will search only for researchers affiliated with the given gridid (a unique value for research institutions)
# as well as the given list of last names to narrow the search down (otherwise we get every person associated with UC which is ~33k)

# The UC Grid ID
GRIDID = 'grid.24827.3b'

# Next, we define the query that will search for researchers based on their last names and the university's Grid ID.
q = """search researchers
        where research_orgs = "{}"
        and last_name in {}
        return researchers"""

# We execute the query and retrieve the results from the Dimensions API. The ```query_iterative``` method is
# used to fetch the data iteratively. This is because otherwise we can only grab up to 1000 results at a time.

researchers_json = dsl.query_iterative(q.format(GRIDID, json.dumps(uniqueLN)))

# `q.format(GRIDID, json.dumps(uniqueLN))`: This formats the query by inserting the GRIDID and the list of last names (uniqueLN).
# The query results are stored in the `researchers_json` variable.

#convert the information we retrieved into a DataFrame object
researchers = researchers_json.as_dataframe()

# This creates a DataFrame called `researchers` that contains the details of the researchers,
# including their names, IDs, and other basic information.

# add a new column (the full name) to make a later step easier
researchers.insert(researchers.columns.get_loc('last_name') + 1,
                   'full_name',
                   researchers['last_name'] + ' ' + researchers['first_name'])

# The `full_name` column is inserted after the `last_name` column.
# The `full_name` is constructed by concatenating the `last_name` and `first_name` for
# each researcher, creating a single string that represents their full name.

In [ ]:
# method 2

# only recomended if you are searching specifically for recent data
# this method loops through the lists of first and last names and searches the database for each one
# it compiles this into a single list before converting it into the rquired pandas dataframe object and removing duplicates

# took 43 minutes and 55 seconds to run
#       tested with ~400 names (this includes preferred and employee name for same people)

# Ensure the two lists are of equal length
if len(empfirst_names) != len(emplast_names):
    raise ValueError("First and last name lists must have the same length!")

# Initialize an empty list to store results
all_researchers = []

GRIDID = 'grid.24827.3b'

# Query loop
for first_name, last_name in zip(empfirst_names, emplast_names):
    # Format the query
    q = """search researchers
            where first_name = "{}"
            and last_name = "{}"
            and research_orgs = "{}"
            return researchers
            limit 1000"""

    # Execute the query iteratively
    researchers_json = dsl.query(q.format(first_name, last_name, GRIDID))

    # Append results to the list
    all_researchers+=researchers_json.researchers
    q = """search researchers
            where first_name = "{}"
            and last_name = "{}"
            and research_orgs is empty
            return researchers
            limit 1000"""
    researchers_json = dsl.query(q.format(first_name, last_name))

    # Append results to the list
    all_researchers+=researchers_json.researchers

pubs_v2 = dimcli.DslDataset.from_publications_list(all_researchers)
researchers = pubs_v2.as_dataframe()
researchers.drop_duplicates("id", inplace=True)

researchers.insert(researchers.columns.get_loc('last_name') + 1,
                   'full_name',
                   researchers['last_name'] + ' ' + researchers['first_name'])

In [ ]:
#to display the entire dataframe run this cell

researchers

### Step 3.2: Apply the Filter to Match Faculty Members
After creating the `full_name` column, we proceed to filter the `researchers` DataFrame by comparing each researcher’s full name to the faculty members' names. This step ensures that only the relevant faculty members are retained, accounting for potential variations in how names may be stored.

In [ ]:
# Create a mask to filter the DataFrame
mask = researchers.apply(
    lambda row: (
        # Check that 'first_name' is a valid string before applying split
        any(
            re.search(rf"^{row['full_name']}\b.*$", name)
            for name in set(empNameAgg + fullName)
        )
    ),
    axis=1
)
# The `apply()` function iterates over each row in the ``researchers` DataFrame and applies the filter logic.
# `re.search()` checks if the combination of the researcher’s last and first name matches any name in `empNameAgg` or `fullName`.
# Using `set(empNameAgg + fullName)` ensures that both lists are merged and duplicates are removed.



# apply the filter to the DataFrame and drop any duplicate entries based on the researcher ID, last name, and first name.
filtered_df = researchers[mask].drop_duplicates(subset=['id', 'last_name', 'first_name'], ignore_index=True)

# `researchers[mask]` filters the rows that match the condition specified in the mask.
# `drop_duplicates()` removes any rows with duplicate researcher IDs, ensuring that each researcher is listed only once.
# `ignore_index=True` resets the index after dropping duplicates.

In [ ]:
#display the data thats been filtered

filtered_df

## Step 4: Retrieving and sorting information from our filtered dataframe

### We can group the ids by the first and last name columns to combine most cases of multiple ids per a person

The following is assuming we used method 1 of grabbing researchers

In [ ]:
researchers_combined = filtered_df.groupby(['last_name', 'first_name', 'full_name'], as_index=False).agg({
                                            'id': list  # Combine ids into a list
                                          })
# Display the result
researchers_combined

By indexing or slicing with `.loc` and `.iloc`, we can combine rows of data manually that were missed or grab only the info associated with a single person (including the alternate names).

In [ ]:
#Example for combining 2 rows that are the same person under different first names
# Get the single row from the DataFrame
single_researcher = researchers_combined.loc[[4], ['first_name', 'last_name', 'id']]

# Append the new ID (from the 5th row) to the list in 'id'
single_researcher['id'].iloc[0].extend(researchers_combined['id'].iloc[5])

In [ ]:
# Display the updated single_researcher
single_researcher

Some people have multiple researcher IDs so this needs manually verified that they are the same or different people


## This retrieves information on the publications by the authors in the filtered_df

All researcher ids

In [ ]:
#use this for a list of all the ids for all of our researchers
researchers_ids = filtered_df['id']
researchers_ids

r_id = list(filtered_df['id'])
r_id

Alternatively create a list for the specific ids you wish to search

In [ ]:
import itertools

# use this if you are searching a single researcher using the above info
r_id = list(itertools.chain.from_iterable(single_researcher['id']))
r_id

In [ ]:
# Adjust the number of researchers IDs per query: so to ensure we never hit the 1000 records limit per query
# this is doable by slicing the r_id list (mainly needed if you are grabbing all the data)
#     you can also change the query to grab the data iteratively:
#         data = dsl.query_iterative(q.format(json.dumps(r_id),json.dumps(years)))


# adjust the years here if you want to look at years different from the earlier list (2018-2024)
#years = []

q = """search publications
       where researchers.id in {}
       and year in {}
       return publications
       limit 1000"""

#change the value of r_id or use a different variable to adjust which person is being looked at
data = dsl.query(q.format(json.dumps(r_id),json.dumps(years)))
data_df = data.as_dataframe()

Lets add a new column to the publication's dataframe.

The following cell will extract all the associated researcher ids with the given publications and add them to the data

The way it does this is to duplicate each row for each discovered researcher_id in the author column so that there is 1 or more rows for each publication dependent on the number of researchers.

In [ ]:
# Extract nested researcher_ids
nested_researcher_ids = [
    [author["researcher_id"] for author in author_list]
    for author_list in data_df["authors"]
]

# Flatten the nested researcher_ids into a single list
flattened_researcher_ids = [researcher_id for ids in nested_researcher_ids for researcher_id in ids]

# Create a new DataFrame with expanded rows
expanded_df = data_df.loc[data_df.index.repeat([len(ids) for ids in nested_researcher_ids])].reset_index(drop=True)

# Add the `r_id` column at the correct position after `id`
expanded_df.insert(
    data_df.columns.get_loc("id") + 1,  # Position after 'id'
    "r_id",
    flattened_researcher_ids
)

# Drop the original authors column (optional)
#expanded_df = expanded_df.drop(columns=["authors"])

expanded_df

In [ ]:
expanded_df[['id', 'r_id', 'title', 'type', 'year']]

In [ ]:
# this lets us get only the researcher ids sorted by publication id
# you can alternatly inverse the "id" and 'r_id' to get each publication grouped by the researcher_id
expanded_df.groupby(["r_id"]).agg({'id': list  # Combine ids into a list
                                          })